# LangGraph Stream Parser - Jupyter Display Demo

This notebook demonstrates the value of `JupyterDisplay` for visualizing LangGraph agent streams.

## The Problem

When streaming from a LangGraph agent, the raw output is:
- **Verbose** - Nested dictionaries with metadata you don't care about
- **Hard to follow** - Tool calls, messages, and interrupts all mixed together
- **Not real-time friendly** - Output floods the cell instead of updating in place

## The Solution

`JupyterDisplay` provides:
- **Clean panels** - Content, tools, and interrupts in organized boxes
- **Live updates** - Updates in place instead of flooding output
- **Tool lifecycle** - See tools transition from running → success/error with timing
- **Prominent interrupts** - Human-in-the-loop requests are impossible to miss

## Setup

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from agent import agent

---
## Before: Raw Streaming Output

This is what you get with vanilla `graph.stream()` - a flood of nested dictionaries:

In [3]:
# Raw streaming - hard to follow!
for chunk in agent.stream(
    dict(messages=["What files are in the current directory?"]), 
    config=dict(configurable=dict(thread_id="raw-demo")),
    stream_mode="updates"
):
    print(chunk)
    print("---")

{'PatchToolCallsMiddleware.before_agent': {'messages': Overwrite(value=[HumanMessage(content='What files are in the current directory?', additional_kwargs={}, response_metadata={}, id='6032e894-e8e5-4882-a123-fffee0730ae6')])}}
---
{'SummarizationMiddleware.before_model': None}
---{'model': {'messages': [AIMessage(content=[{'text': "I'll list the files in the current directory for you.", 'type': 'text'}, {'id': 'toolu_019ifLxGo51EssTxhscxURuB', 'input': {'path': '/'}, 'name': 'ls', 'type': 'tool_use'}], additional_kwargs={}, response_metadata={'id': 'msg_01UyW3e5Meaz2oTUSkY2bn5g', 'model': 'claude-sonnet-4-5-20250929', 'stop_reason': 'tool_use', 'stop_sequence': None, 'usage': {'cache_creation': {'ephemeral_1h_input_tokens': 0, 'ephemeral_5m_input_tokens': 6258}, 'cache_creation_input_tokens': 6258, 'cache_read_input_tokens': 0, 'input_tokens': 3, 'output_tokens': 64, 'server_tool_use': None, 'service_tier': 'standard'}, 'model_name': 'claude-sonnet-4-5-20250929', 'model_provider': 'an

---
## After: JupyterDisplay

Same stream, but with clean, live-updating panels:

In [3]:
from langgraph_stream_parser.adapters.jupyter import JupyterDisplay

In [4]:
display = JupyterDisplay()

display.stream(
    agent.stream(
        dict(messages=["What files are in the current directory?"]), 
        config=dict(configurable=dict(thread_id="display-demo")),
        stream_mode="updates"
    )
)

╭─────────────────────────────────── tools ────────────────────────────────────╮
│ OK ls path=/ 37ms                                                            │
╰──────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────── assistant ──────────────────────────────────╮
│ The current directory contains:                                              │
│                                                                              │
│ 1. **`.ipynb_checkpoints/`** - Directory for Jupyter notebook checkpoints    │
│ 2. **`__pycache__/`** - Directory for Python bytecode cache files            │
│ 3. **`agent.py`** - A Python script file                                     │
│ 4. **`jupyter_display.py`** - A Python script file                           │
│ 5. **`jupyter_example.ipynb`** - A Jupyter notebook file                     │
│                                                                              │
│ Would you like me to explore any of these files in more detail?              │
╰──────────────────────────────────────────────────────────────────────────────╯

done

In [5]:
display = JupyterDisplay()

display.stream(
    agent.stream(
        dict(messages=["what's in the third file of this folder? Use the think_tool and write_todos. Use the bash tool for ls"]), 
        config=dict(configurable=dict(thread_id="display-demo2")),
        stream_mode="updates"
    )
)

OK think_tool reflection=The user wants me to find ... 22ms

OK write_todos todos=[{'content': 'Use bash tool... 25ms

reflection: The user wants me to find the third file in "this folder" but 
specifically wants me to use the bash tool for ls instead of the ls function. 
Since no specific path was provided, I should assume they me...

todos: 
  ▶ Use bash tool to list files in the current directory
  ○ Identify the third file from the output
  ○ Read the contents of the third file

╭───────────────────────────────── assistant ──────────────────────────────────╮
│ I'll help you explore the folder and find the third file using the bash tool │
│ for ls. Let me start by using the think_tool and write_todos as requested.   │
╰──────────────────────────────────────────────────────────────────────────────╯

... bash command=ls -la

╭───────────────────────────────── assistant ──────────────────────────────────╮
│ Now let me use the bash tool to list files:                                  │
╰──────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────── interrupt ──────────────────────────────────╮
│ bash(command=ls -la)                                                         │
│ options: approve/edit/reject                                                 │
╰──────────────────────────────────────────────────────────────────────────────╯

done

---
## Key Features

### 1. Tool Lifecycle Tracking

Tools show their status in real-time:
- `⏳ ...` - Running
- `✅ OK` - Success (with execution time)
- `❌ ERR` - Error

### 2. Interrupt Handling

When the agent needs approval (like for `bash` commands), you get a prominent red panel showing:
- The tool requesting approval
- The arguments it wants to use
- Allowed decisions (approve/reject/edit)

### 3. Content Accumulation

Assistant responses stream smoothly in a blue panel instead of chunk-by-chunk spam.

---
## Manual Event Processing

For more control, you can process events individually:

In [8]:
from langgraph_stream_parser import StreamParser
from langgraph_stream_parser.events import InterruptEvent, ContentEvent, ToolCallStartEvent

parser = StreamParser()
display = JupyterDisplay()

for event in parser.parse(
    agent.stream(
        dict(messages=["Count Python files in this directory"]),
        config=dict(configurable=dict(thread_id="manual-demo")),
        stream_mode="updates"
    )
):
    display.update(event)
    
    # You can also handle events programmatically
    if isinstance(event, InterruptEvent):
        print(f"\n[Interrupt detected - {len(event.action_requests)} action(s) need approval]")

╭─────────────────────────────────── tools ────────────────────────────────────╮
│ OK ls path=/ 18ms                                                            │
│ OK glob pattern=**/*.py 9ms                                                  │
╰──────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────── assistant ──────────────────────────────────╮
│ I'll help you count the Python files in this directory. Let me start by      │
│ exploring the current directory structure.                                   │
│ There are **3 Python files** in this directory (including subdirectories):   │
│                                                                              │
│ 1. `/agent.py`                                                               │
│ 2. `/jupyter_display.py`                                                     │
│ 3. `/.ipynb_checkpoints/agent-checkpoint.py`                                 │
│                                                                              │
│ If you want to exclude checkpoint files (which are typically auto-generated  │
│ backups), there are **2 main Python files** in the root directory:           │
│ - `agent.py`                                                                 │
│ - `jupyter_display.py`                                                       │
╰──────────────────────────────────────────────────────────────────────────────╯

done

---
## Configuration Options

In [9]:
# Customize the display
display = JupyterDisplay(
    show_timestamps=False,     # Show/hide timestamps
    show_tool_args=True,       # Show tool arguments in the status table
    max_content_preview=200,   # Max characters for extraction previews
)

---
## Summary

| Raw Streaming | JupyterDisplay |
|---------------|----------------|
| Nested dicts flood output | Clean, organized panels |
| Tool status buried in data | Visual tool lifecycle (⏳→✅/❌) |
| Interrupts easy to miss | Prominent red interrupt panel |
| Content comes in chunks | Smooth content accumulation |
| No timing information | Execution time per tool |